# 02_create_splits — Split único reproducible para todos los baselines

**Objetivo:** crear **una sola vez** los splits train/val estratificados y guardarlos en `data/splits/` para que **todos los baselines** (rule-based, TF-IDF, transformer) usen **exactamente los mismos ejemplos** y sean **comparables**.

**Outputs:**
- `data/splits/train_indices.csv` (índices para train)
- `data/splits/val_indices.csv` (índices para val)
- `data/splits/dataset_base.csv` (dataset normalizado base con índices)

**Reproducibilidad:** `random_state=42` fijo.

In [9]:
# === Paths / Globals ===
from pathlib import Path
import pandas as pd
import unicodedata
from sklearn.model_selection import train_test_split

BASE_PATH = Path.cwd()
if BASE_PATH.name == "notebooks":
    BASE_PATH = BASE_PATH.parent

DATA_PATH = BASE_PATH / "data"
SPLITS_PATH = DATA_PATH / "splits"
SPLITS_PATH.mkdir(exist_ok=True)

# Priorizar ips_clean.csv (generado por 01_eda) sobre ips_raw.csv
INPUT_FILE = DATA_PATH / 'ips_clean.csv'
if not INPUT_FILE.exists():
    INPUT_FILE = DATA_PATH / 'ips_raw.csv'
    print("⚠️ No se encontró ips_clean.csv, usando ips_raw.csv")
    print("   Recomendación: ejecutar 01_eda_understanding.ipynb primero")
    if not INPUT_FILE.exists():
        raise FileNotFoundError(f"No se encontró ni ips_clean.csv ni ips_raw.csv en {DATA_PATH}")

print("📥 INPUT_FILE:", INPUT_FILE)
print("📁 SPLITS_PATH:", SPLITS_PATH)

📥 INPUT_FILE: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/ips_clean.csv
📁 SPLITS_PATH: /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits


## 1) Carga y normalización base (sin preprocesamiento de texto)

In [10]:
# Columnas esperadas
TEXT_COL = "texto"
LABEL_COL = "etiqueta"

def _guess_text_col(df):
    if TEXT_COL in df.columns: 
        return TEXT_COL
    for c in ['texto','Motivo Consulta','original_motivo_consulta','text']:
        if c in df.columns: return c
    for c in df.columns:
        if df[c].dtype == 'O': return c
    raise ValueError("No se encontró columna de texto.")

def _guess_label_col(df):
    if LABEL_COL in df.columns: 
        return LABEL_COL
    for c in ['etiqueta','Tipo','label','target','y','clase']:
        if c in df.columns: return c
    return None

def _norm_label_bin(s):
    """Normaliza etiquetas a 'ansiedad' o 'depresion'"""
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s).encode("ascii","ignore").decode("ascii")
    return {'depresivo':'depresion'}.get(s, s)

# Cargar
df_raw = pd.read_csv(INPUT_FILE)
text_col = _guess_text_col(df_raw)
label_col = _guess_label_col(df_raw)

if label_col is None:
    raise ValueError("Se requiere columna de etiquetas.")

# Filtrar y normalizar (si viene de ips_raw, ya debería estar limpio si viene de ips_clean)
df = df_raw.dropna(subset=[text_col, label_col]).copy()
df[label_col] = df[label_col].map(_norm_label_bin)
df = df[df[label_col].isin(['ansiedad','depresion'])].copy()

# Eliminar duplicados en texto (por si viene de ips_raw)
n_before = len(df)
df = df.drop_duplicates(subset=[text_col]).copy()
n_after = len(df)
if n_before > n_after:
    print(f"⚠️ Eliminados {n_before - n_after} duplicados (usar ips_clean.csv para evitar esto)")

# Resetear índice y crear ID único
df = df.reset_index(drop=True)
df['row_id'] = df.index

print(f"✅ Dataset cargado: {len(df)} ejemplos")
print(f"Distribución:\n{df[label_col].value_counts()}")

✅ Dataset cargado: 3125 ejemplos
Distribución:
etiqueta
depresion    2200
ansiedad      925
Name: count, dtype: int64


## 2) Split estratificado (80/20) con semilla fija

In [11]:
RANDOM_STATE = 42
TEST_SIZE = 0.2

train_idx, val_idx = train_test_split(
    df.index,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=df[label_col]
)

print(f"Train: {len(train_idx)} | Val: {len(val_idx)}")
print(f"Train distribución:\n{df.loc[train_idx, label_col].value_counts()}")
print(f"Val distribución:\n{df.loc[val_idx, label_col].value_counts()}")

Train: 2500 | Val: 625
Train distribución:
etiqueta
depresion    1760
ansiedad      740
Name: count, dtype: int64
Val distribución:
etiqueta
depresion    440
ansiedad     185
Name: count, dtype: int64


## 3) Guardar splits y dataset base

In [12]:
# Guardar índices
pd.DataFrame({'row_id': train_idx}).to_csv(SPLITS_PATH/'train_indices.csv', index=False)
pd.DataFrame({'row_id': val_idx}).to_csv(SPLITS_PATH/'val_indices.csv', index=False)

# Guardar dataset base (con texto original y etiqueta normalizada)
df_base = df[['row_id', text_col, label_col]].copy()
df_base.to_csv(SPLITS_PATH/'dataset_base.csv', index=False, encoding='utf-8')

print("\n✅ Archivos guardados:")
print(f" - {SPLITS_PATH/'train_indices.csv'}")
print(f" - {SPLITS_PATH/'val_indices.csv'}")
print(f" - {SPLITS_PATH/'dataset_base.csv'}")
print(f"\n📊 Origen: {INPUT_FILE.name}")
print(f"🔒 Ahora todos los baselines deben usar estos splits.")


✅ Archivos guardados:
 - /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/train_indices.csv
 - /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/val_indices.csv
 - /Users/manuelnunez/Projects/psych-phenotyping-paraguay/data/splits/dataset_base.csv

📊 Origen: ips_clean.csv
🔒 Ahora todos los baselines deben usar estos splits.
